# 🔬 End-to-End Research-Grade ML Classification Pipeline

**Sections**
1. Environment & Reproducibility
2. Data Ingestion & Exploratory Data Analysis (EDA)
3. Feature Engineering & Preprocessing
4. Model Training & Hyperparameter Optimisation (HPO)
5. Evaluation — Cross-Validation, Curves, Permutation Importance
6. Final Model Summary

> Dataset: *Breast Cancer Wisconsin (Diagnostic)* — a well-understood binary classification benchmark.  
> Swap `load_dataset()` in §2 with your own `pd.read_csv()` call to adapt instantly.

## 1 · Environment & Reproducibility

In [1]:
# ── Imports ──────────────────────────────────────────────────────────────────
import warnings, time, json
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from scipy import stats

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import (
    train_test_split, StratifiedKFold, cross_validate,
    RandomizedSearchCV, learning_curve
)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectFromModel, mutual_info_classif

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
)
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import (
    accuracy_score, f1_score, roc_auc_score,
    classification_report, confusion_matrix,
    RocCurveDisplay, PrecisionRecallDisplay,
    ConfusionMatrixDisplay, average_precision_score
)
from sklearn.inspection import permutation_importance

# ── Global config ─────────────────────────────────────────────────────────────
SEED       = 42
TEST_SIZE  = 0.20
CV_FOLDS   = 5
N_ITER_HPO = 40          # RandomizedSearchCV iterations per model
FIGSIZE    = (14, 5)

np.random.seed(SEED)
sns.set_theme(style="whitegrid", palette="muted", font_scale=1.1)
plt.rcParams.update({"figure.dpi": 120, "axes.spines.top": False, "axes.spines.right": False})

print(f"✅  Environment ready  |  NumPy {np.__version__}  |  Scikit-learn from sklearn")

✅  Environment ready  |  NumPy 2.4.2  |  Scikit-learn from sklearn


## 2 · Data Ingestion & EDA

We load the Breast Cancer Wisconsin dataset and perform systematic exploratory analysis covering:
- Shape, dtypes, missing-value audit
- Class balance
- Univariate distributions
- Pairwise correlation heatmap
- Statistical outlier detection (Z-score)

In [ ]:
# ── 2.1  Load ─────────────────────────────────────────────────────────────────
def load_dataset():
    """Returns X (DataFrame) and y (Series). Swap this function for your data."""
    raw = load_breast_cancer(as_frame=True)
    X = raw.data.copy()
    y = raw.target.rename("target")          # 0 = malignant, 1 = benign
    return X, y

X, y = load_dataset()
TARGET_NAMES = {0: "Malignant", 1: "Benign"}
NUM_FEATURES = X.select_dtypes(include="number").columns.tolist()
CAT_FEATURES = X.select_dtypes(exclude="number").columns.tolist()

print(f"Dataset shape : {X.shape}")
print(f"Target        : {y.value_counts().to_dict()}")
print(f"Numeric cols  : {len(NUM_FEATURES)}  |  Categorical cols : {len(CAT_FEATURES)}")
X.head(3)

In [ ]:
# ── 2.2  Missing value & dtype audit ──────────────────────────────────────────
audit = pd.DataFrame({
    "dtype"    : X.dtypes,
    "missing"  : X.isna().sum(),
    "missing %" : (X.isna().mean() * 100).round(2),
    "unique"   : X.nunique(),
    "mean"     : X.mean().round(3),
    "std"      : X.std().round(3),
})
print(audit.to_string())

In [ ]:
# ── 2.3  Class balance ────────────────────────────────────────────────────────
fig, axes = plt.subplots(1, 2, figsize=FIGSIZE)

counts = y.map(TARGET_NAMES).value_counts()
axes[0].bar(counts.index, counts.values, color=["#e74c3c", "#2ecc71"], edgecolor="white", linewidth=1.5)
axes[0].set_title("Class Distribution", fontweight="bold")
axes[0].set_ylabel("Count")
for i, v in enumerate(counts.values):
    axes[0].text(i, v + 2, str(v), ha="center", fontweight="bold")

axes[1].pie(counts.values, labels=counts.index, autopct="%1.1f%%",
            colors=["#e74c3c", "#2ecc71"], startangle=90,
            wedgeprops=dict(edgecolor="white", linewidth=2))
axes[1].set_title("Class Proportion", fontweight="bold")

plt.suptitle("Target Variable Analysis", fontsize=14, fontweight="bold", y=1.01)
plt.tight_layout()
plt.show()
print(f"Imbalance ratio (majority/minority): {counts.max()/counts.min():.2f}")

In [ ]:
# ── 2.4  Feature distributions (top-15 by variance) ───────────────────────────
top15 = X[NUM_FEATURES].var().nlargest(15).index.tolist()

fig, axes = plt.subplots(3, 5, figsize=(18, 9))
axes = axes.flatten()
for i, col in enumerate(top15):
    for label, color in [(0, "#e74c3c"), (1, "#2ecc71")]:
        axes[i].hist(X.loc[y == label, col], bins=25, alpha=0.6,
                     color=color, label=TARGET_NAMES[label], density=True)
    axes[i].set_title(col, fontsize=9)
    axes[i].set_yticks([])

axes[0].legend(fontsize=8)
fig.suptitle("Feature Distributions by Class (Top 15 by Variance)", fontsize=13, fontweight="bold")
plt.tight_layout()
plt.show()

In [ ]:
# ── 2.5  Correlation heatmap ──────────────────────────────────────────────────
corr = X[NUM_FEATURES].corr()
mask = np.triu(np.ones_like(corr, dtype=bool))

fig, ax = plt.subplots(figsize=(16, 13))
sns.heatmap(corr, mask=mask, cmap="RdYlGn", center=0, vmin=-1, vmax=1,
            annot=False, linewidths=0.3, ax=ax, cbar_kws={"shrink": 0.8})
ax.set_title("Pearson Correlation Matrix", fontsize=14, fontweight="bold", pad=12)
plt.tight_layout()
plt.show()

# Highly correlated pairs
high_corr = (corr.abs()
             .where(np.tril(np.ones(corr.shape), k=-1).astype(bool))
             .stack()
             .sort_values(ascending=False))
print("Top 10 correlated pairs:")
print(high_corr.head(10).to_string())

In [ ]:
# ── 2.6  Outlier detection (Z-score) ─────────────────────────────────────────
z_scores = np.abs(stats.zscore(X[NUM_FEATURES]))
outlier_mask = (z_scores > 3).any(axis=1)
print(f"Samples with |z| > 3 in any feature: {outlier_mask.sum()} ({outlier_mask.mean()*100:.1f}%)")

fig, ax = plt.subplots(figsize=FIGSIZE)
outlier_counts = (z_scores > 3).sum(axis=0)
outlier_counts.sort_values(ascending=False).head(15).plot(
    kind="bar", ax=ax, color="#e67e22", edgecolor="white")
ax.set_title("Features with Most Outliers (|Z-score| > 3)", fontweight="bold")
ax.set_ylabel("# Outlier Samples")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

## 3 · Feature Engineering & Preprocessing

Pipeline:
1. **Imputation** — median for numerics, most-frequent for categoricals
2. **Scaling** — RobustScaler (outlier-resistant)
3. **Feature selection** — Mutual information + SelectFromModel (ExtraTrees)
4. **Train / test split** — stratified 80/20

In [ ]:
# ── 3.1  Train/test split ──────────────────────────────────────────────────────
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=SEED, stratify=y
)
print(f"Train: {X_train.shape}  |  Test: {X_test.shape}")
print(f"Train class balance: {y_train.value_counts().to_dict()}")
print(f"Test  class balance: {y_test.value_counts().to_dict()}")

In [ ]:
# ── 3.2  Mutual Information ────────────────────────────────────────────────────
mi = mutual_info_classif(X_train[NUM_FEATURES].fillna(X_train[NUM_FEATURES].median()),
                          y_train, random_state=SEED)
mi_series = pd.Series(mi, index=NUM_FEATURES).sort_values(ascending=False)

fig, ax = plt.subplots(figsize=FIGSIZE)
mi_series.head(20).plot(kind="bar", ax=ax, color="#3498db", edgecolor="white")
ax.set_title("Mutual Information with Target (Top 20 Features)", fontweight="bold")
ax.set_ylabel("MI Score")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

In [ ]:
# ── 3.3  Preprocessing pipeline ──────────────────────────────────────────────
numeric_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler",  RobustScaler()),
])

if CAT_FEATURES:
    from sklearn.preprocessing import OrdinalEncoder
    categorical_transformer = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("encoder", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)),
    ])
    preprocessor = ColumnTransformer([
        ("num", numeric_transformer, NUM_FEATURES),
        ("cat", categorical_transformer, CAT_FEATURES),
    ])
else:
    preprocessor = ColumnTransformer([
        ("num", numeric_transformer, NUM_FEATURES),
    ])

# Fit on train, transform both splits
X_train_prep = preprocessor.fit_transform(X_train)
X_test_prep  = preprocessor.transform(X_test)
print(f"Preprocessed train shape: {X_train_prep.shape}")

In [ ]:
# ── 3.4  Feature selection via ExtraTrees importance ──────────────────────────
selector = SelectFromModel(
    ExtraTreesClassifier(n_estimators=200, random_state=SEED),
    threshold="mean"
)
selector.fit(X_train_prep, y_train)

X_train_sel = selector.transform(X_train_prep)
X_test_sel  = selector.transform(X_test_prep)

n_kept = X_train_sel.shape[1]
n_orig = X_train_prep.shape[1]
print(f"Features kept: {n_kept} / {n_orig}  ({n_kept/n_orig*100:.0f}%)")

# Feature importance from selector
importances = selector.estimator_.feature_importances_
feat_names  = [f"f{i}" for i in range(n_orig)]   # generic names post-transform
importance_df = (pd.DataFrame({"feature": feat_names, "importance": importances})
                   .sort_values("importance", ascending=False).head(20))

fig, ax = plt.subplots(figsize=FIGSIZE)
ax.barh(importance_df["feature"][::-1], importance_df["importance"][::-1],
        color="#9b59b6", edgecolor="white")
ax.set_title("ExtraTrees Feature Importances (Top 20)", fontweight="bold")
ax.set_xlabel("Mean Decrease in Impurity")
plt.tight_layout()
plt.show()

## 4 · Model Training & Hyperparameter Optimisation

We train five model families with **RandomizedSearchCV** (stratified CV, optimising ROC-AUC):

| Model | Search Space |
|---|---|
| Logistic Regression | C, penalty, solver |
| Random Forest | n_estimators, max_depth, min_samples_split |
| Gradient Boosting | n_estimators, learning_rate, max_depth |
| Extra Trees | n_estimators, max_features, min_samples_leaf |
| SVM | C, kernel, gamma |

In [ ]:
# ── 4.1  Search spaces ────────────────────────────────────────────────────────
from scipy.stats import loguniform, randint

SEARCH_SPACES = {
    "LogisticRegression": {
        "model": LogisticRegression(max_iter=2000, random_state=SEED),
        "params": {
            "C"      : loguniform(1e-3, 1e3),
            "penalty": ["l2"],
            "solver" : ["lbfgs", "saga"],
        }
    },
    "RandomForest": {
        "model": RandomForestClassifier(random_state=SEED, n_jobs=-1),
        "params": {
            "n_estimators"    : randint(50, 400),
            "max_depth"       : [None, 5, 10, 20],
            "min_samples_split": randint(2, 20),
            "max_features"    : ["sqrt", "log2", 0.5],
        }
    },
    "GradientBoosting": {
        "model": GradientBoostingClassifier(random_state=SEED),
        "params": {
            "n_estimators" : randint(50, 300),
            "learning_rate": loguniform(0.01, 0.5),
            "max_depth"    : randint(2, 8),
            "subsample"    : [0.7, 0.8, 0.9, 1.0],
        }
    },
    "ExtraTrees": {
        "model": ExtraTreesClassifier(random_state=SEED, n_jobs=-1),
        "params": {
            "n_estimators"    : randint(50, 400),
            "max_features"    : ["sqrt", "log2", 0.5, 0.8],
            "min_samples_leaf": randint(1, 10),
        }
    },
    "SVM": {
        "model": SVC(probability=True, random_state=SEED),
        "params": {
            "C"     : loguniform(0.1, 100),
            "kernel": ["rbf", "poly"],
            "gamma" : ["scale", "auto"],
        }
    },
}

cv = StratifiedKFold(n_splits=CV_FOLDS, shuffle=True, random_state=SEED)
print(f"HPO config: {N_ITER_HPO} iterations × {CV_FOLDS}-fold CV per model")

In [ ]:
# ── 4.2  Run HPO ──────────────────────────────────────────────────────────────
hpo_results = {}

for name, cfg in SEARCH_SPACES.items():
    t0 = time.time()
    search = RandomizedSearchCV(
        cfg["model"], cfg["params"],
        n_iter=N_ITER_HPO, cv=cv,
        scoring="roc_auc", refit=True,
        random_state=SEED, n_jobs=-1, verbose=0
    )
    search.fit(X_train_sel, y_train)
    elapsed = time.time() - t0

    hpo_results[name] = {
        "search"      : search,
        "best_model"  : search.best_estimator_,
        "best_params" : search.best_params_,
        "cv_roc_auc"  : search.best_score_,
        "elapsed_s"   : elapsed,
    }
    print(f"  {name:<22}  CV ROC-AUC = {search.best_score_:.4f}  ({elapsed:.1f}s)")

print("\n✅  HPO complete for all models")

In [ ]:
# ── 4.3  HPO convergence – mean test score vs iteration ───────────────────────
fig, axes = plt.subplots(1, len(hpo_results), figsize=(18, 4), sharey=False)

for ax, (name, res) in zip(axes, hpo_results.items()):
    cv_df = pd.DataFrame(res["search"].cv_results_)
    ax.plot(cv_df["mean_test_score"], "o-", ms=3, lw=1.5, color="#2980b9", alpha=0.8)
    ax.axhline(res["cv_roc_auc"], color="#e74c3c", ls="--", lw=1.5, label=f"Best={res['cv_roc_auc']:.4f}")
    ax.set_title(name, fontsize=9, fontweight="bold")
    ax.set_xlabel("Iteration")
    ax.set_ylabel("ROC-AUC")
    ax.legend(fontsize=8)

fig.suptitle("HPO Convergence (CV ROC-AUC per Iteration)", fontsize=13, fontweight="bold")
plt.tight_layout()
plt.show()

## 5 · Evaluation

### 5.1  Cross-validated leaderboard
### 5.2  Test-set metrics
### 5.3  Confusion matrices
### 5.4  ROC & Precision-Recall curves
### 5.5  Learning curves (bias-variance)
### 5.6  Permutation importance (model-agnostic, SHAP-equivalent)

In [ ]:
# ── 5.1  CV leaderboard ────────────────────────────────────────────────────────
records = []
for name, res in hpo_results.items():
    cv_out = cross_validate(
        res["best_model"], X_train_sel, y_train, cv=cv,
        scoring=["roc_auc", "f1", "accuracy"], n_jobs=-1
    )
    records.append({
        "Model"       : name,
        "CV ROC-AUC"  : f"{cv_out['test_roc_auc'].mean():.4f} ± {cv_out['test_roc_auc'].std():.4f}",
        "CV F1"       : f"{cv_out['test_f1'].mean():.4f} ± {cv_out['test_f1'].std():.4f}",
        "CV Accuracy" : f"{cv_out['test_accuracy'].mean():.4f} ± {cv_out['test_accuracy'].std():.4f}",
        "HPO (s)"     : f"{res['elapsed_s']:.1f}",
    })
    res["cv_detailed"] = cv_out

leaderboard = pd.DataFrame(records).sort_values("CV ROC-AUC", ascending=False)
leaderboard.index = range(1, len(leaderboard)+1)
print("=" * 80)
print("Cross-Validated Leaderboard")
print("=" * 80)
print(leaderboard.to_string())

In [ ]:
# ── 5.2  Test-set metrics ─────────────────────────────────────────────────────
test_records = []
for name, res in hpo_results.items():
    m = res["best_model"]
    y_pred  = m.predict(X_test_sel)
    y_proba = m.predict_proba(X_test_sel)[:, 1]
    test_records.append({
        "Model"     : name,
        "Accuracy"  : accuracy_score(y_test, y_pred),
        "F1"        : f1_score(y_test, y_pred),
        "ROC-AUC"   : roc_auc_score(y_test, y_proba),
        "Avg-Prec"  : average_precision_score(y_test, y_proba),
    })

test_df = pd.DataFrame(test_records).sort_values("ROC-AUC", ascending=False)
test_df.index = range(1, len(test_df)+1)
test_df[["Accuracy","F1","ROC-AUC","Avg-Prec"]] = test_df[["Accuracy","F1","ROC-AUC","Avg-Prec"]].round(4)
print(test_df.to_string())

# Best model
best_name = test_df.iloc[0]["Model"]
best_model = hpo_results[best_name]["best_model"]
print(f"\n🏆  Best model on test set: {best_name}")

In [ ]:
# ── 5.3  Confusion matrices ───────────────────────────────────────────────────
fig, axes = plt.subplots(1, len(hpo_results), figsize=(20, 4))

for ax, (name, res) in zip(axes, hpo_results.items()):
    y_pred = res["best_model"].predict(X_test_sel)
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", ax=ax,
                xticklabels=["Mal", "Ben"], yticklabels=["Mal", "Ben"],
                cbar=False, linewidths=0.5)
    ax.set_title(f"{name}\nAcc={accuracy_score(y_test,y_pred):.3f}", fontsize=9)
    ax.set_xlabel("Predicted")
    ax.set_ylabel("Actual")

fig.suptitle("Confusion Matrices (Test Set)", fontsize=13, fontweight="bold")
plt.tight_layout()
plt.show()

In [ ]:
# ── 5.4  ROC & Precision-Recall curves ────────────────────────────────────────
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
colors = ["#2980b9","#27ae60","#e74c3c","#9b59b6","#e67e22"]

for (name, res), color in zip(hpo_results.items(), colors):
    y_proba = res["best_model"].predict_proba(X_test_sel)[:, 1]
    RocCurveDisplay.from_predictions(y_test, y_proba, name=name, ax=axes[0], color=color, alpha=0.85)
    PrecisionRecallDisplay.from_predictions(y_test, y_proba, name=name, ax=axes[1], color=color, alpha=0.85)

axes[0].plot([0,1],[0,1],"k--",lw=1)
axes[0].set_title("ROC Curves (Test Set)", fontweight="bold")
axes[1].set_title("Precision-Recall Curves (Test Set)", fontweight="bold")
axes[0].legend(loc="lower right", fontsize=8)
axes[1].legend(loc="upper right", fontsize=8)
plt.tight_layout()
plt.show()

In [ ]:
# ── 5.5  Learning curves (best model) ─────────────────────────────────────────
train_sizes, train_scores, val_scores = learning_curve(
    best_model, X_train_sel, y_train,
    cv=cv, scoring="roc_auc",
    train_sizes=np.linspace(0.1, 1.0, 10),
    n_jobs=-1, shuffle=True, random_state=SEED
)

fig, ax = plt.subplots(figsize=FIGSIZE)
ax.fill_between(train_sizes,
                train_scores.mean(1) - train_scores.std(1),
                train_scores.mean(1) + train_scores.std(1), alpha=0.15, color="#2980b9")
ax.fill_between(train_sizes,
                val_scores.mean(1)   - val_scores.std(1),
                val_scores.mean(1)   + val_scores.std(1),   alpha=0.15, color="#e74c3c")
ax.plot(train_sizes, train_scores.mean(1), "o-", color="#2980b9", label="Train ROC-AUC")
ax.plot(train_sizes, val_scores.mean(1),   "o-", color="#e74c3c", label="Val ROC-AUC")
ax.set_xlabel("Training samples")
ax.set_ylabel("ROC-AUC")
ax.set_title(f"Learning Curve — {best_name}", fontweight="bold")
ax.legend()
ax.set_ylim(0.7, 1.02)
plt.tight_layout()
plt.show()

gap = train_scores.mean(1)[-1] - val_scores.mean(1)[-1]
print(f"Train-Val gap at full data: {gap:.4f}  → {'⚠ slight overfit' if gap > 0.03 else '✅ well-generalised'}")

In [ ]:
# ── 5.6  Permutation importance (model-agnostic) ──────────────────────────────
perm = permutation_importance(
    best_model, X_test_sel, y_test,
    n_repeats=30, random_state=SEED,
    scoring="roc_auc", n_jobs=-1
)

perm_df = pd.DataFrame({
    "feature"   : [f"f{i}" for i in range(X_test_sel.shape[1])],
    "mean"      : perm.importances_mean,
    "std"       : perm.importances_std,
}).sort_values("mean", ascending=False).head(20)

fig, ax = plt.subplots(figsize=FIGSIZE)
ax.barh(perm_df["feature"][::-1], perm_df["mean"][::-1],
        xerr=perm_df["std"][::-1], color="#1abc9c", edgecolor="white",
        error_kw=dict(ecolor="gray", capsize=3))
ax.set_xlabel("Mean decrease in ROC-AUC (± std)")
ax.set_title(f"Permutation Importance — {best_name}  (n_repeats=30)", fontweight="bold")
plt.tight_layout()
plt.show()

print("Top 5 most important features:")
print(perm_df.head(5)[["feature","mean","std"]].to_string(index=False))

In [ ]:
# ── 5.7  Detailed classification report (best model) ──────────────────────────
y_pred = best_model.predict(X_test_sel)
print(f"Classification Report — {best_name}")
print("=" * 55)
print(classification_report(y_test, y_pred, target_names=list(TARGET_NAMES.values())))

## 6 · Final Model Summary

In [ ]:
# ── 6.1  Summary card ─────────────────────────────────────────────────────────
y_pred  = best_model.predict(X_test_sel)
y_proba = best_model.predict_proba(X_test_sel)[:, 1]

summary = {
    "Best Model"        : best_name,
    "Best Hyperparams"  : hpo_results[best_name]["best_params"],
    "Test Accuracy"     : f"{accuracy_score(y_test, y_pred):.4f}",
    "Test F1"           : f"{f1_score(y_test, y_pred):.4f}",
    "Test ROC-AUC"      : f"{roc_auc_score(y_test, y_proba):.4f}",
    "Test Avg Precision": f"{average_precision_score(y_test, y_proba):.4f}",
    "Features (orig)"   : X.shape[1],
    "Features (selected)": int(X_train_sel.shape[1]),
    "Train samples"     : len(X_train),
    "Test samples"      : len(X_test),
    "CV Folds"          : CV_FOLDS,
    "HPO Iterations"    : N_ITER_HPO,
}

print("╔══════════════════════════════════════════════════════════╗")
print("║           FINAL PIPELINE SUMMARY                        ║")
print("╠══════════════════════════════════════════════════════════╣")
for k, v in summary.items():
    if k == "Best Hyperparams":
        print(f"║  {k:<28}: (see below)               ║")
    else:
        print(f"║  {k:<28}: {str(v):<28} ║")
print("╚══════════════════════════════════════════════════════════╝")
print("\nBest hyperparameters:")
for k, v in hpo_results[best_name]["best_params"].items():
    print(f"  {k}: {v}")

---
### Pipeline Checklist

| Step | Status |
|---|---|
| Data ingestion | ✅ |
| Missing value audit | ✅ |
| EDA (distributions, correlation, outliers) | ✅ |
| Median imputation + RobustScaler | ✅ |
| Mutual information feature analysis | ✅ |
| ExtraTrees feature selection | ✅ |
| 5-model HPO (RandomizedSearchCV) | ✅ |
| Stratified K-Fold cross-validation | ✅ |
| Confusion matrices | ✅ |
| ROC & Precision-Recall curves | ✅ |
| Learning curves | ✅ |
| Permutation importance | ✅ |

> **To adapt to your own data:** replace `load_dataset()` in §2 with `pd.read_csv(...)`, update `NUM_FEATURES` / `CAT_FEATURES`, and adjust `TARGET_NAMES`.